# USGS Water Use by County in 2015

#### The [U.S. Geological Survey's National Water-Use Science Project](https://water.usgs.gov/watuse/), compiles and reports water-use data for the nation.

-  Water-Use reports and data are published at 5 year intervals starting in 1950. The USGS published the most recent report [Estimated use of water in the United States in 2015](https://pubs.er.usgs.gov/publication/cir1441) in June, 2018. 
-  Along with this report the USGS released [Estimated Use of Water in the United States County-Level Data for 2015](https://www.sciencebase.gov/catalog/item/get/5af3311be4b0da30c1b245d8). We will use this dataset to get a sense for public and private drinking water-use in Vermont on the county-scale and how it compares to the rest of the Continental United States.

In [10]:
# Load necessary libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import sciencebasepy

#### Access relevant data
- Water use data from sciencebase using [sciencebasepy](https://github.com/usgs/sciencebasepy) module.
- US County spatial data from 2015 from [US Census](https://www.census.gov/geo/maps-data/data/cbf/cbf_counties.html)

In [11]:
sb = sciencebasepy.SbSession()

dataset_id = '5af3311be4b0da30c1b245d8'

# Get a public item.  No need to log in.
item_json = sb.get_item(dataset_id)

# Find csv path in attached files
url = [file for file in item_json['files'] if 'csv' in file['name']][0]['url']

# Read csv into dataframe
water_use_2015 = pd.read_csv(url, header = 1)

In [12]:
water_use_2015.head()

,STATE,STATEFIPS,COUNTY,COUNTYFIPS,FIPS,YEAR,TP-TotPop,PS-GWPop,PS-SWPop,PS-TOPop,...,TO-WGWTo,TO-WSWFr,TO-WSWSa,TO-WSWTo,TO-WFrTo,TO-WSaTo,TO-Wtotl,TO-CUsFrPartial,TO-CUsSaPartial,TO-CUTotPartial
0,AL,1,Autauga County,1,1001,2015,55.347,--,--,48.998,...,10.69,45.72,0.0,45.72,56.41,0.0,56.41,13.18,0.0,13.18
1,AL,1,Baldwin County,3,1003,2015,203.709,--,--,174.907,...,73.38,11.32,0.0,11.32,84.70,0.0,84.70,58.28,0.0,58.28
2,AL,1,Barbour County,5,1005,2015,26.489,--,--,24.587,...,6.06,2.96,0.0,2.96,9.02,0.0,9.02,2.96,0.0,2.96
3,AL,1,Bibb County,7,1007,2015,22.583,--,--,20.947,...,5.67,0.20,0.0,0.20,5.87,0.0,5.87,0.19,0.0,0.19
4,AL,1,Blount County,9,1009,2015,57.673,--,--,44.815,...,4.64,55.43,0.0,55.43,60.07,0.0,60.07,1.08,0.0,1.08


The column names are somewhat cryptic so we will also create a dictionary from the Data Dictionary in the excel file for reference

In [15]:
# The column names are somewhat cryptic so we will also create a dictionary from the Data Dictionary in the excel file 
# for reference
excel_url = [file for file in item_json['files'] if 'xlsx' in file['name']][0]['url']
water_use_dict = (pd.read_excel(excel_url, sheet_name = 'DataDictionary')).set_index('Column Tag').T.to_dict('list')

In [16]:
water_use_dict

{'STATE': ['State postal abbreviation'],
 'STATEFIPS': ['State FIPS code'],
 'COUNTY': ['County name'],
 'COUNTYFIPS': ['County FIPS code'],
 'FIPS': ['Concatenated State-county FIPS code'],
 'YEAR': ['Year of data=2015'],
 'TP-TotPop': ['Total population of county, in thousands'],
 'PS-GWPop': ['Public Supply, population served by groundwater, in thousands'],
 'PS-SWPop': ['Public Supply, population served by surface water, in thousands'],
 'PS-TOPop': ['Public Supply, total population served, in thousands'],
 'PS-WGWFr': ['Public Supply,  groundwater withdrawals, fresh, in Mgal/d'],
 'PS-WGWSa': ['Public Supply, groundwater withdrawals, saline, in Mgal/d'],
 'PS-WGWTo': ['Public Supply, groundwater withdrawals, total, in Mgal/d'],
 'PS-WSWFr': ['Public Supply, surface-water withdrawals, fresh, in Mgal/d'],
 'PS-WSWSa': ['Public Supply, surface-water withdrawals, saline, in Mgal/d'],
 'PS-WSWTo': ['Public Supply, surface-water withdrawals, total, in Mgal/d'],
 'PS-WFrTo': ['Public Sup

Use geopandas to create geodataframes of US counties in 2015
- For the moment I will do this using a locally hosted shapefile downloaded from the link above. 
- An improvement would be to read it directly from the Census FTP site following [Agaidus Python Notebook - Reading Zipped Shapefiles](https://github.com/agaidus/census_data_extraction/blob/master/Reading_Zipped_Shapefiles.ipynb)
- Or from the github repo

In [23]:
# Read in shapefiles
counties_shp_path = r"C:\Users\colin.dowey\OneDrive - State of Vermont\MyFiles\GIS\GeneralGISFiles\shapefiles\US_County_500k_2015\cb_2015_us_county_500k.shp"
counties = gpd.read_file(counties_shp_path)
VTcounties = counties[counties['STATEFP'] == '50']

#### Join water-use data to counties geodataframe
- As you will see this join is not perfect and there is some mismatch between the County FIPS codes used to join
- For Vermont this is not an issue and in the future I plan on following through on some of the changed county FIPS codes
- However, if you are working in South Dakota, for example, this is something you will need to address
- Both datasets are from the same year but clearly there is still some mismatch


In [ ]:
# Convert FIPS to text in Water Use Data, if only four characters, then add leading zero
water_use_2015['FIPS'] = water_use_2015['FIPS'].astype(str)
water_use_2015['FIPS'] = [('0' + a) if len(a) == 4 else a for a in water_use_2015['FIPS']]